In [4]:
import requests
import csv
import time
import json
import sys
import imp
import pandas as pd
imp.reload(sys)




<module 'sys' (built-in)>

In [6]:
TOKEN = 'BQBOAGr4vgzbGhWhcKRkl4ImFL78TEHqknFdhVHAHZMbxuO3FbF-226PNb7yuIGPcSZ_XPcyBNZNDpD7-hhq1RebaQ2paJs4dsm-E1XArf53mhEdjYYhlm5HfIEJKz1MxjU6zNK08vBXAhdXuB1gUB2pQJ2D8T8L-weTCPPfBs5uQ31c'

In [8]:
print('start..')


def main():
    
    queries = ['1995','1996','1997','1998','1999','2000','2001',
              '2002','2003','2004','2005','2006','2007','2008',
              '2009','2010','2011','2012','2013','2014','2015','2016','2017']
              
    #queries = ['2017']
    # Query and request from API are different!
    # Number of track query need to make
    num_tracks_per_query = 10000

    for query in queries:
        
        ltrack = []
        song_ids = []
        artist_ids = []
        album_ids = []
        
        audioF = []
        artist_data = []
        album_data = []

        
        col1 = [   'popularity',
                   'song_id',     'artist_id',     'album_id', 
                   'song_name',   'artist_name',   'album_name', 
                   'explicit',    'disc_number',   'track_number']
        
        
        col2 =  [  'song_id', 'uri',
                   'tempo', 'type',
                   'key', 'loudness',
                   'mode', 'speechiness',
                   'liveness', 'valence',
                   'danceability', 'energy',
                   'track_href', 'analysis_url',
                   'duration_ms', 'time_signature',
                   'acousticness', 'instrumentalness' ]
        
        col3 =  [  'artist_id',  'artist_genres',  'artist_popularity']
        
        col4 =  [  'album_id',  'album_genres',   'album_popularity',  'album_release_date']
        
        n = 0 
        idx = 0
        
        while idx < num_tracks_per_query:  
            
            API_search_request(query, 'track', 50, idx, ltrack, song_ids, artist_ids, album_ids)   
            n +=1
            print(('\n>> this is No '+ str(n) + ' search End '))
            idx += 50 
            # Limit API requests to at most 3ish calls / second
            time.sleep(0.3)                                     
        
        print(len(song_ids))
        ## spotify API "search" option vs here track/audiofeature query
        for idx in range(0, len(song_ids), 50):
            API_get_audio_feature(song_ids[idx: idx+50], audioF)
            time.sleep(0.3)
        
        for idx in range(0, len(artist_ids), 50):
            API_get_artists(artist_ids[idx: idx+50], artist_data)
            time.sleep(0.3)
        
        for idx in range(0, len(album_ids), 20):
            API_get_albums(album_ids[idx: idx+20], album_data)
            time.sleep(0.3)    
        
        
        df1 = pd.DataFrame(ltrack, columns=col1)
        
        df2 = pd.DataFrame(audioF, columns=col2) 
        
        
        df3 = pd.DataFrame(artist_data, columns=col3)
        
        df4 = pd.DataFrame(album_data, columns=col4)
        
        df = df1.merge(df2, on='song_id', how='outer').merge(df3, on='artist_id', how='outer').merge(
             df4, on='album_id', how='outer')
        
        filename = query + '.csv'                      
        
        df.to_csv(filename, sep='\t')
        
        print ('finish')
        print (query)

def API_search_request(keywords, search_type, results_limit, results_offset, ltrack, song_ids, artist_ids, album_ids):
    print("API_search_request")
    off = str(results_offset)
    lim = str(results_limit)

    url = 'https://api.spotify.com/v1/search?q=year:'+ keywords +'&type=' + search_type +'&offset='+ off +'&limit=' + lim
    headers = {'Authorization': "Bearer {}".format(TOKEN)}
    r = requests.get(url, headers=headers)
    print(r)
    if r: 
        j = r.json()
    else:
        return r


    litem = j['tracks']['items']
    #print(len(ll))
    try:
        for l in litem:
        
            if l['id'] not in song_ids:
                song_ids.append( l['id'] )

            if l['artists'][0]['id'] not in artist_ids:
                artist_ids.append( l['artists'][0]['id'] )

            if l['album']['id'] not in album_ids:
                album_ids.append(  l['album']['id'] )
        
        
            k =   [  l['popularity'],
        
                     l['id'], 
                     l['artists'][0]['id'],
                     l['album']['id'],

                     l['name'],
                     l['artists'][0]['name'],
                     l['album']['name'],

                     l['explicit'], 
                     l['disc_number'],
                     l['track_number']]
        
            ltrack.append(k)
    except:
         ValueError
      
   # f.close()
    #return j


def API_get_audio_feature(songids, audioF):
    
    #print(songids)
    #print '>> call art several'
    track_ids = ','.join(songids)

    url = 'https://api.spotify.com/v1/audio-features?ids=' + track_ids  
    ## access_token will expire soon
    headers = {'Authorization': "Bearer {}".format(TOKEN)}

    #access_token = (  f'Bearer {}BQC85HmfPsvXmNqGXcXp6BcKWhEySEuillUvd42QsF8Zj9rG3TAHzOUQ9IF-dMnJiphDitn9cBM-i0gBFkyS26gzmEA_lADpvs_HaYy3sgo53IEzySXKE37xLv6jqZSVuRBQ8z317DuyiaSqKapUEqRgdoIGAwUZlDkCxFhRC_p7L67X')
    
    
    r = requests.get(url, headers=headers)
    
    if r: 
        j = r.json()
    else:
        return r
    
    # print(j)
    ll = j['audio_features']

    try:

        for l in ll:
            k =  [  l['id'],l['uri'],
                    l['tempo'],l['type'],
                    l['key'],l['loudness'],
                    l['mode'],l['speechiness'],
                    l['liveness'],l['valence'],
                    l['danceability'],l['energy'],
                    l['track_href'],l['analysis_url'],
                    l['duration_ms'],l['time_signature'],
                    l['acousticness'],l['instrumentalness'] ]

            audioF.append(k)
        
    except:
        ValueError
    
        

    #return j

def API_get_artists(artist_ids, artist_data):

    art_ids = ','.join(artist_ids)
    headers = {'Authorization': "Bearer {}".format(TOKEN)}

    url = 'https://api.spotify.com/v1/artists?ids=' + art_ids

    r = requests.get(url, headers=headers)

    if r:
        j = r.json()
    else:
        return r

    
    ll = j['artists']

    try:
        for l in ll:
        
            k = [  l['id'], 
                   l['genres'],
                   l['popularity'] ]

            artist_data.append(k)
    
    except:
        ValueError
    


def API_get_albums(album_ids, album_data):
   

    alb_ids = ','.join(album_ids)

    url = 'https://api.spotify.com/v1/albums?ids=' + alb_ids
    headers = {'Authorization': "Bearer {}".format(TOKEN)}

    r = requests.get(url, headers=headers)

    if r:
        j = r.json()
    else:
        return r


    ll = j['albums']
    
    try:
        for l in ll:
            k = [  l['id'], 
                   l['genres'],
                   l['popularity'],
                   l['release_date'] ]

            album_data.append(k)
    
    except:
        ValueError


main()        

start..
API_search_request
<Response [200]>

>> this is No 1 search End 
API_search_request
<Response [200]>

>> this is No 2 search End 
API_search_request
<Response [200]>

>> this is No 3 search End 
API_search_request
<Response [200]>

>> this is No 4 search End 
API_search_request
<Response [200]>

>> this is No 5 search End 
API_search_request
<Response [200]>

>> this is No 6 search End 
API_search_request
<Response [200]>

>> this is No 7 search End 
API_search_request
<Response [200]>

>> this is No 8 search End 
API_search_request
<Response [200]>

>> this is No 9 search End 
API_search_request
<Response [200]>

>> this is No 10 search End 
API_search_request
<Response [200]>

>> this is No 11 search End 
API_search_request
<Response [200]>

>> this is No 12 search End 
API_search_request
<Response [200]>

>> this is No 13 search End 
API_search_request
<Response [200]>

>> this is No 14 search End 
API_search_request
<Response [200]>

>> this is No 15 search End 
API_search_

API_search_request
<Response [200]>

>> this is No 125 search End 
API_search_request
<Response [200]>

>> this is No 126 search End 
API_search_request
<Response [200]>

>> this is No 127 search End 
API_search_request
<Response [200]>

>> this is No 128 search End 
API_search_request
<Response [200]>

>> this is No 129 search End 
API_search_request
<Response [200]>

>> this is No 130 search End 
API_search_request
<Response [200]>

>> this is No 131 search End 
API_search_request
<Response [200]>

>> this is No 132 search End 
API_search_request
<Response [200]>

>> this is No 133 search End 
API_search_request
<Response [200]>

>> this is No 134 search End 
API_search_request
<Response [200]>

>> this is No 135 search End 
API_search_request
<Response [200]>

>> this is No 136 search End 
API_search_request
<Response [200]>

>> this is No 137 search End 
API_search_request
<Response [200]>

>> this is No 138 search End 
API_search_request
<Response [200]>

>> this is No 139 search 

API_search_request
<Response [200]>

>> this is No 48 search End 
API_search_request
<Response [200]>

>> this is No 49 search End 
API_search_request
<Response [200]>

>> this is No 50 search End 
API_search_request
<Response [200]>

>> this is No 51 search End 
API_search_request
<Response [200]>

>> this is No 52 search End 
API_search_request
<Response [200]>

>> this is No 53 search End 
API_search_request
<Response [200]>

>> this is No 54 search End 
API_search_request
<Response [200]>

>> this is No 55 search End 
API_search_request
<Response [200]>

>> this is No 56 search End 
API_search_request
<Response [200]>

>> this is No 57 search End 
API_search_request
<Response [200]>

>> this is No 58 search End 
API_search_request
<Response [200]>

>> this is No 59 search End 
API_search_request
<Response [200]>

>> this is No 60 search End 
API_search_request
<Response [200]>

>> this is No 61 search End 
API_search_request
<Response [200]>

>> this is No 62 search End 
API_search

<Response [200]>

>> this is No 171 search End 
API_search_request
<Response [200]>

>> this is No 172 search End 
API_search_request
<Response [200]>

>> this is No 173 search End 
API_search_request
<Response [200]>

>> this is No 174 search End 
API_search_request
<Response [200]>

>> this is No 175 search End 
API_search_request
<Response [200]>

>> this is No 176 search End 
API_search_request
<Response [200]>

>> this is No 177 search End 
API_search_request
<Response [200]>

>> this is No 178 search End 
API_search_request
<Response [200]>

>> this is No 179 search End 
API_search_request
<Response [200]>

>> this is No 180 search End 
API_search_request
<Response [200]>

>> this is No 181 search End 
API_search_request
<Response [200]>

>> this is No 182 search End 
API_search_request
<Response [200]>

>> this is No 183 search End 
API_search_request
<Response [200]>

>> this is No 184 search End 
API_search_request
<Response [200]>

>> this is No 185 search End 
API_search_req

API_search_request
<Response [200]>

>> this is No 95 search End 
API_search_request
<Response [200]>

>> this is No 96 search End 
API_search_request
<Response [200]>

>> this is No 97 search End 
API_search_request
<Response [200]>

>> this is No 98 search End 
API_search_request
<Response [200]>

>> this is No 99 search End 
API_search_request
<Response [200]>

>> this is No 100 search End 
API_search_request
<Response [200]>

>> this is No 101 search End 
API_search_request
<Response [200]>

>> this is No 102 search End 
API_search_request
<Response [200]>

>> this is No 103 search End 
API_search_request
<Response [200]>

>> this is No 104 search End 
API_search_request
<Response [200]>

>> this is No 105 search End 
API_search_request
<Response [200]>

>> this is No 106 search End 
API_search_request
<Response [200]>

>> this is No 107 search End 
API_search_request
<Response [200]>

>> this is No 108 search End 
API_search_request
<Response [200]>

>> this is No 109 search End 


...

API_search_request
<Response [401]>

>> this is No 182 search End 
API_search_request
<Response [401]>

>> this is No 183 search End 
API_search_request
<Response [401]>

>> this is No 184 search End 
API_search_request
<Response [401]>

>> this is No 185 search End 
API_search_request
<Response [401]>

>> this is No 186 search End 
API_search_request
<Response [401]>

>> this is No 187 search End 
API_search_request
<Response [401]>

>> this is No 188 search End 
API_search_request
<Response [401]>

>> this is No 189 search End 
API_search_request
<Response [401]>

>> this is No 190 search End 
API_search_request
<Response [401]>

>> this is No 191 search End 
API_search_request
<Response [401]>

>> this is No 192 search End 
API_search_request
<Response [401]>

>> this is No 193 search End 
API_search_request
<Response [401]>

>> this is No 194 search End 
API_search_request
<Response [401]>

>> this is No 195 search End 
API_search_request
<Response [401]>

>> this is No 196 search 

API_search_request
<Response [401]>

>> this is No 106 search End 
API_search_request
<Response [401]>

>> this is No 107 search End 
API_search_request
<Response [401]>

>> this is No 108 search End 
API_search_request
<Response [401]>

>> this is No 109 search End 
API_search_request
<Response [401]>

>> this is No 110 search End 
API_search_request
<Response [401]>

>> this is No 111 search End 
API_search_request
<Response [401]>

>> this is No 112 search End 
API_search_request
<Response [401]>

>> this is No 113 search End 
API_search_request
<Response [401]>

>> this is No 114 search End 
API_search_request
<Response [401]>

>> this is No 115 search End 
API_search_request
<Response [401]>

>> this is No 116 search End 
API_search_request
<Response [401]>

>> this is No 117 search End 
API_search_request
<Response [401]>

>> this is No 118 search End 
API_search_request
<Response [401]>

>> this is No 119 search End 
API_search_request
<Response [401]>

>> this is No 120 search 

API_search_request
<Response [401]>

>> this is No 29 search End 
API_search_request
<Response [401]>

>> this is No 30 search End 
API_search_request
<Response [401]>

>> this is No 31 search End 
API_search_request
<Response [401]>

>> this is No 32 search End 
API_search_request
<Response [401]>

>> this is No 33 search End 
API_search_request
<Response [401]>

>> this is No 34 search End 
API_search_request
<Response [401]>

>> this is No 35 search End 
API_search_request
<Response [401]>

>> this is No 36 search End 
API_search_request
<Response [401]>

>> this is No 37 search End 
API_search_request
<Response [401]>

>> this is No 38 search End 
API_search_request
<Response [401]>

>> this is No 39 search End 
API_search_request
<Response [401]>

>> this is No 40 search End 
API_search_request
<Response [401]>

>> this is No 41 search End 
API_search_request
<Response [401]>

>> this is No 42 search End 
API_search_request
<Response [401]>

>> this is No 43 search End 
API_search

API_search_request
<Response [401]>

>> this is No 153 search End 
API_search_request
<Response [401]>

>> this is No 154 search End 
API_search_request
<Response [401]>

>> this is No 155 search End 
API_search_request
<Response [401]>

>> this is No 156 search End 
API_search_request
<Response [401]>

>> this is No 157 search End 
API_search_request
<Response [401]>

>> this is No 158 search End 
API_search_request
<Response [401]>

>> this is No 159 search End 
API_search_request
<Response [401]>

>> this is No 160 search End 
API_search_request
<Response [401]>

>> this is No 161 search End 
API_search_request
<Response [401]>

>> this is No 162 search End 
API_search_request
<Response [401]>

>> this is No 163 search End 
API_search_request
<Response [401]>

>> this is No 164 search End 
API_search_request
<Response [401]>

>> this is No 165 search End 
API_search_request
<Response [401]>

>> this is No 166 search End 
API_search_request
<Response [401]>

>> this is No 167 search 

KeyboardInterrupt: 